### Incorporating requested changes prior to the IWAAs data release

This notebook is for examining some of the issues with the delivered IWAAs files and preparing the files for delivery based on requests from USGS 4/8/2024.

In [1]:
import os
import sys
import time

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

In [2]:
Ten_Year = False
Forty_Year = True

# Input files
if Ten_Year:
    #in_nc = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY2010_2021_nan.nc'
    in_nc = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY2010_2021.nc'
    wlt_file = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/CONUS_HUC12_WB_2D_top_soil_20091001_20210930.nc'
elif Forty_Year:
    #in_nc = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_nan.nc'
    in_nc = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_2.nc'

# Output files
if Ten_Year:
    out_file = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY2010_2021_nan2.nc'
elif Forty_Year:
    out_file = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_nan2.nc'

In [3]:
# Set the nodata value
nodata_value = float(-9999)

### Read latest IWAAs Water Budget NetCDF File

In [4]:
ds = xr.open_dataset(in_nc)
#ds.load()

### 1) Drop unecessary dimension on `yrmo` variable

This is request # 4 in the 4/8/2024 request list.

In [5]:
yrmo_dim_len = [len(ds[dim]) for dim in ds['yrmo'].dims]
if len(ds['yrmo'].dims) > 1 and 1 in yrmo_dim_len:
   remove_dims = [dim for dim in ds['yrmo'].dims if len(ds[dim])==1]
   print('Removing dimension(s) {0} from variable "yrmo".'.format(remove_dims))
   ds['yrmo'] = ds['yrmo'].squeeze()
ds

Removing dimension(s) ['yrmo_index'] from variable "yrmo".


<xarray.Dataset> Size: 2GB
Dimensions:                    (huc_id: 86617, time: 516)
Coordinates:
  * huc_id                     (huc_id) |S12 1MB b'010100020101' ... b'181002...
  * time                       (time) datetime64[ns] 4kB 1979-10-01 ... 2022-...
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 179MB ...
    Snowfall                   (huc_id, time) float32 179MB ...
    Surfaceflow                (huc_id, time) float32 179MB ...
    Baseflow                   (huc_id, time) float32 179MB ...
    ET                         (huc_id, time) float32 179MB ...
    SWE                        (huc_id, time) float32 179MB ...
    ...                         ...
    Precip                     (huc_id, time) float32 179MB ...
    LandFraction               (huc_id) float32 346kB ...
    total_gridded_area         (huc_id) float32 346kB ...
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 179MB ...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 179MB ...
    yrmo                       (time) <U7 14kB ...
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### 2) Expand the `huc_id` dimension to include all 86,733 HUC12 units

Read in the HUC12 data from GDB

In [6]:
%%time

# HUC-12 GPKG Geometry File
in_HUCs = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/HUCs/HUC12.gpkg'
layer_name = 'WBDHU12_CONUS'

# Print info
#! ogrinfo {in_HUCs} {layer_name} -so

gdf = gpd.read_file(in_HUCs, layer='WBDHU12_CONUS', ignore_geometry=True)
gdf

CPU times: user 1.51 s, sys: 40.9 s, total: 42.4 s
Wall time: 46.1 s


,AREASQKM,STATES,HUC12,TOHUC
0,51.25,AL,031401030101,031401030102
1,149.86,AL,031401030102,031401030103
2,105.27,AL,031401030103,031401030302
3,104.41,AL,031401030104,031401030302
4,137.70,AL,031401030201,031401030203
...,...,...,...,...
86730,135.12,CN,040101010102,040101010103
86731,83.94,CN,040101011501,041800000200
86732,1639.80,"CN,MI",042400000102,042400000200
86733,4242.18,"CN,MI",042400000101,042400000200


#### Check for consistency with PRMS basins

Sydney provided a netCDF file with the HUC12 IDs from NHM/PRMS on 4/18/2024. We will evaluate this list against ours to ensure consistency.

In [7]:
nhm_prms_basins = r'/glade/derecho/scratch/ksampson/USGS/NHM_PRMS/list_of_hu12_ids_NHMPRMS.nc'

# Open the dataset
ds_prms = xr.open_dataset(nhm_prms_basins)

# Obtain array of HUC12 IDs from PRMS
prms_huc12s = ds_prms['huc_id'].data

# Get the size of each array
print(prms_huc12s.shape)
print(ds['huc_id'].data.shape)

(86733,)
(86617,)


In [8]:
prms_not_in_nwm = ds_prms['huc_id'].data[~np.in1d(ds_prms['huc_id'].data, ds['huc_id'].data)]
print(prms_not_in_nwm.shape)

nwm_not_in_prms = ds['huc_id'].data[~np.in1d(ds['huc_id'].data, ds_prms['huc_id'].data)]
print(nwm_not_in_prms.shape)

(116,)
(0,)


In [9]:
# Subset the geodataframe to just the values not pesent in the PRMS file
missing_from_nwm = gdf[gdf['HUC12'].isin(prms_not_in_nwm)]
assert len(missing_from_nwm) == len(prms_not_in_nwm)
missing_from_nwm

,AREASQKM,STATES,HUC12,TOHUC
8891,246.77,CA,180500020802,180500021002
9913,244.13,CA,180500050600,OCEAN
14268,149.42,GA,030701060505,OCEAN
15134,114.39,GA,030702030204,OCEAN
15139,169.25,GA,030702030403,OCEAN
...,...,...,...,...
85406,182.36,RI,010900050406,OCEAN
85407,628.26,"NY,RI",020302020804,OCEAN
85409,231.87,FL,031401020900,OCEAN
85410,424.58,FL,031401050600,OCEAN


#### Create new empty DataSet that includes the missing HUC12 IDs

In [10]:
# Create an empty dataset matching the source datset structure, and subset size to what we need
add_ds = xr.full_like(ds, nodata_value).isel({'huc_id':slice(0,len(missing_from_nwm),None)})

# Chaange the HUC12 coordinates to the ones we want
add_ds['huc_id'] = missing_from_nwm['HUC12'].to_numpy()

# Add in data for the static variables

# Add in yrmo from source dataset
add_ds['yrmo'] = ds['yrmo']

# Add in Catchment Area
add_ds['CatchmentArea'][:] = missing_from_nwm['AREASQKM'].to_numpy()

# Add in LandFraction
add_ds['LandFraction'][:] = 0.0

# View result
add_ds

<xarray.Dataset> Size: 3MB
Dimensions:                    (huc_id: 116, time: 516)
Coordinates:
  * huc_id                     (huc_id) object 928B '180500020802' ... '17100...
  * time                       (time) datetime64[ns] 4kB 1979-10-01 ... 2022-...
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 239kB -9.999e+03 ... -9...
    Snowfall                   (huc_id, time) float32 239kB -9.999e+03 ... -9...
    Surfaceflow                (huc_id, time) float32 239kB -9.999e+03 ... -9...
    Baseflow                   (huc_id, time) float32 239kB -9.999e+03 ... -9...
    ET                         (huc_id, time) float32 239kB -9.999e+03 ... -9...
    SWE                        (huc_id, time) float32 239kB -9.999e+03 ... -9...
    ...                         ...
    Precip                     (huc_id, time) float32 239kB -9.999e+03 ... -9...
    LandFraction               (huc_id) float32 464B 0.0 0.0 0.0 ... 0.0 0.0 0.0
    total_gridded_area         (huc_id) float32 464B -9.999e+03 ... -9.999e+03
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 239kB -9.999e+03 ... -9...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 239kB -9.999e+03 ... -9...
    yrmo                       (time) <U7 14kB '1979-10' '1979-11' ... '2022-09'
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

#### Combine datasets

This seems to add the 'huc_id' dimension to `yrmo`.

In [11]:
out_ds = xr.merge([ds, add_ds], fill_value=nodata_value, combine_attrs="override", compat='override')
del add_ds
out_ds

<xarray.Dataset> Size: 2GB
Dimensions:                    (huc_id: 86733, time: 516)
Coordinates:
  * huc_id                     (huc_id) object 694kB b'010100020101' ... '180...
  * time                       (time) datetime64[ns] 4kB 1979-10-01 ... 2022-...
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 179MB 98.15 ... -9.999e+03
    Snowfall                   (huc_id, time) float32 179MB 10.53 ... -9.999e+03
    Surfaceflow                (huc_id, time) float32 179MB 5.523 ... -9.999e+03
    Baseflow                   (huc_id, time) float32 179MB 49.09 ... -9.999e+03
    ET                         (huc_id, time) float32 179MB 26.78 ... -9.999e+03
    SWE                        (huc_id, time) float32 179MB 0.03072 ... -9.99...
    ...                         ...
    Precip                     (huc_id, time) float32 179MB 98.22 ... -9.999e+03
    LandFraction               (huc_id) float32 347kB 0.9412 1.0 ... -9.999e+03
    total_gridded_area         (huc_id) float32 347kB 64.0 75.0 ... -9.999e+03
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 179MB 0.3048 ... -9.999...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 179MB 0.6726 ... -9.999...
    yrmo                       (time) <U7 14kB '1979-10' '1979-11' ... '2022-09'
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### 3) Make the WRF-Hydro dataset match the HUC12 order of the NHM/PRMS dataset

In [12]:
%%time

# Change the dtype to conform to the PRMS file
out_ds['huc_id'] = out_ds['huc_id'].astype(prms_huc12s.dtype)

if not out_ds['huc_id'].equals(prms_huc12s):
    print('Not sorted identically. Sorting to match PRMS data.')
    out_ds = out_ds.reindex({'huc_id':prms_huc12s}, copy=True)
assert np.array_equal(out_ds['huc_id'].data, prms_huc12s)
out_ds

Not sorted identically. Sorting to match PRMS data.
CPU times: user 357 ms, sys: 416 ms, total: 773 ms
Wall time: 837 ms


<xarray.Dataset> Size: 2GB
Dimensions:                    (huc_id: 86733, time: 516)
Coordinates:
  * huc_id                     (huc_id) <U12 4MB '010100020101' ... '18050005...
  * time                       (time) datetime64[ns] 4kB 1979-10-01 ... 2022-...
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 179MB 98.15 ... -9.999e+03
    Snowfall                   (huc_id, time) float32 179MB 10.53 ... -9.999e+03
    Surfaceflow                (huc_id, time) float32 179MB 5.523 ... -9.999e+03
    Baseflow                   (huc_id, time) float32 179MB 49.09 ... -9.999e+03
    ET                         (huc_id, time) float32 179MB 26.78 ... -9.999e+03
    SWE                        (huc_id, time) float32 179MB 0.03072 ... -9.99...
    ...                         ...
    Precip                     (huc_id, time) float32 179MB 98.22 ... -9.999e+03
    LandFraction               (huc_id) float32 347kB 0.9412 1.0 ... -9.999e+03
    total_gridded_area         (huc_id) float32 347kB 64.0 67.0 ... -9.999e+03
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 179MB 0.3048 ... -9.999...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 179MB 0.6726 ... -9.999...
    yrmo                       (time) <U7 14kB '1979-10' '1979-11' ... '2022-09'
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### 4) Incorporate the wilting-point adjusted data [ only complete for 10 year dataset ]

In [13]:
if Ten_Year:
    print('Ten year file. Adding wilting point variables.')
    
    # Open the file
    ds_wlt = xr.open_dataset(wlt_file)
    
    # Rename the HUC12 dimension
    ds_wlt = ds_wlt.rename({'WBDHU12': 'huc_id'})
    
    # Change any datatypes necessary to conform to output dataset
    #ds_wlt['huc_id'] = ds_wlt['huc_id'].astype(out_ds['huc_id'].dtype)
    out_ds['huc_id'] = out_ds['huc_id'].astype(ds_wlt['huc_id'].dtype)
    
    # Check for compatibliity along the time dimension
    assert ds_wlt['time'].equals(out_ds['time'])
    
    # Re-sort to match order of basins
    ds_wlt = ds_wlt.reindex({'huc_id':out_ds['huc_id'].data}, copy=True)
    assert ds_wlt['huc_id'].equals(out_ds['huc_id'])
    
    # Add the new variables in
    out_ds = xr.merge([out_ds, ds_wlt], join='left')
    
    # Re-assign data type back to U12
    out_ds['huc_id'] = out_ds['huc_id'].astype(prms_huc12s.dtype)
    
    # Close file
    ds_wlt.close()
    del ds_wlt

elif Forty_Year:
    print('Forty year file already has desired wilting point variables.')
    pass

# Display
out_ds

Forty year file already has desired wilting point variables.


<xarray.Dataset> Size: 2GB
Dimensions:                    (huc_id: 86733, time: 516)
Coordinates:
  * huc_id                     (huc_id) <U12 4MB '010100020101' ... '18050005...
  * time                       (time) datetime64[ns] 4kB 1979-10-01 ... 2022-...
Data variables: (12/17)
    PrecipLand                 (huc_id, time) float32 179MB 98.15 ... -9.999e+03
    Snowfall                   (huc_id, time) float32 179MB 10.53 ... -9.999e+03
    Surfaceflow                (huc_id, time) float32 179MB 5.523 ... -9.999e+03
    Baseflow                   (huc_id, time) float32 179MB 49.09 ... -9.999e+03
    ET                         (huc_id, time) float32 179MB 26.78 ... -9.999e+03
    SWE                        (huc_id, time) float32 179MB 0.03072 ... -9.99...
    ...                         ...
    Precip                     (huc_id, time) float32 179MB 98.22 ... -9.999e+03
    LandFraction               (huc_id) float32 347kB 0.9412 1.0 ... -9.999e+03
    total_gridded_area         (huc_id) float32 347kB 64.0 67.0 ... -9.999e+03
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 179MB 0.3048 ... -9.999...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 179MB 0.6726 ... -9.999...
    yrmo                       (time) <U7 14kB '1979-10' '1979-11' ... '2022-09'
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### 5) Enforce a sort order on the variables in the dataset, for consistency between 10-year and 40-year files.

In [14]:
# Build list of variables, sorted by 1D, then 2D alphabetical
sorted_varlist = ['yrmo', 'CatchmentArea', 'LandFraction', 'total_gridded_area']

# Build list of all 2D+ variables, sorted alphabetically no matter the case
sorted_varlist2 = [item for item in list(out_ds.data_vars) if item not in sorted_varlist]
sorted_varlist2.sort(key=str.casefold)

# Add the lists together
sorted_varlist += sorted_varlist2
assert len(list(out_ds.data_vars)) == len(sorted_varlist)
print('Found {0} variables. Sorted by number of dimensions and then alphabetically'.format(len(sorted_varlist)))
      
# Sort the variables in the dataset
out_ds = out_ds[sorted_varlist]
out_ds

Found 17 variables. Sorted by number of dimensions and then alphabetically


<xarray.Dataset> Size: 2GB
Dimensions:                    (time: 516, huc_id: 86733)
Coordinates:
  * huc_id                     (huc_id) <U12 4MB '010100020101' ... '18050005...
  * time                       (time) datetime64[ns] 4kB 1979-10-01 ... 2022-...
Data variables: (12/17)
    yrmo                       (time) <U7 14kB '1979-10' '1979-11' ... '2022-09'
    CatchmentArea              (huc_id) float32 347kB 74.25 75.25 ... -9.999e+03
    LandFraction               (huc_id) float32 347kB 0.9412 1.0 ... -9.999e+03
    total_gridded_area         (huc_id) float32 347kB 64.0 67.0 ... -9.999e+03
    avgSOILM_wltadj_depthmean  (huc_id, time) float32 179MB 0.3048 ... -9.999...
    avgSOILSAT_wltadj_top1     (huc_id, time) float32 179MB 0.6726 ... -9.999...
    ...                         ...
    Recharge                   (huc_id, time) float32 179MB 54.64 ... -9.999e+03
    Snowfall                   (huc_id, time) float32 179MB 10.53 ... -9.999e+03
    SoilSat                    (huc_id, time) float32 179MB 0.7824 ... -9.999...
    SoilWater                  (huc_id, time) float32 179MB 776.4 ... -9.999e+03
    Surfaceflow                (huc_id, time) float32 179MB 5.523 ... -9.999e+03
    SWE                        (huc_id, time) float32 179MB 0.03072 ... -9.99...
Attributes:
    title:        HUC12 monthly water budget components from WRF-Hydro IWAA v1.0
    institution:  NCAR
    source1:      https://github.com/NOAA-Big-Data-Program/bdp-data-docs/blob...
    source2:      https://www.sciencebase.gov/catalog/file/get/60cb5edfd34e86...
    history:      A. Dugger, Tue Mar 14 20:37:45 2023

### 6) Fix NoData Issues

A decision was made to make all NaN values consistent between 10-year and 40-year Water Budget component files, using -9999.0 as the _FillValue in the netCDF files, and adding descirptions to variable attributes to identify what NaN means in each variable.

#### Fix variable attributes and encodings

In [15]:
output_encoding = {}
for data_var in out_ds.data_vars:
    print(data_var)

    # Fix nodata description attribute
    if data_var in ['PrecipLand', 
                    'Snowfall', 
                    'ET', 
                    'SWE', 
                    'SoilWater', 
                    'SoilSat', 
                    'Recharge', 
                    'Precip', 
                    'LandFraction', 
                    'total_gridded_area', 
                    'avgSOILM_wltadj_depthmean', 
                    'avgSOILSAT_wltadj_top1']:
        out_ds[data_var].attrs['nodata_description'] = 'HUC12 contains no land cells'
    elif data_var in ['CatchmentArea']:
        out_ds[data_var].attrs['nodata_description'] = 'HUC12 contains no WRF-Hydro catchment polygons'
    elif data_var in ['Surfaceflow', 
                      'Baseflow', 
                      'GWStore']:
        out_ds[data_var].attrs['nodata_description'] = 'HUC12 contain no WRF-Hydro flowlines.'
    else:
        continue
    print('\tnodata_description: {0}'.format(out_ds[data_var].attrs['nodata_description']))

    # Change NaN to nodata value
    nodata_mask = out_ds[data_var].isnull().data
    print('\tFound {0} nodata values'.format(nodata_mask.sum()))
    #display(out_ds[data_var].data[nodata_mask])
    out_ds[data_var].data[nodata_mask] = nodata_value
    #display(out_ds[data_var].data[nodata_mask])
    
    # Variable encoding
    out_ds[data_var].encoding['_FillValue'] = nodata_value
    output_encoding[data_var] = {'_FillValue':nodata_value}

    # Remove redundant missing value encoding
    if 'missing_value' in out_ds[data_var].encoding:
        del out_ds[data_var].encoding['missing_value']
    print('\t'.format(out_ds[data_var].encoding['_FillValue']))

yrmo
CatchmentArea
	nodata_description: HUC12 contains no WRF-Hydro catchment polygons
	Found 3343 nodata values
	
LandFraction
	nodata_description: HUC12 contains no land cells
	Found 13 nodata values
	
total_gridded_area
	nodata_description: HUC12 contains no land cells
	Found 13 nodata values
	
avgSOILM_wltadj_depthmean
	nodata_description: HUC12 contains no land cells
	Found 6708 nodata values
	
avgSOILSAT_wltadj_top1
	nodata_description: HUC12 contains no land cells
	Found 6708 nodata values
	
Baseflow
	nodata_description: HUC12 contain no WRF-Hydro flowlines.
	Found 1746660 nodata values
	
ET
	nodata_description: HUC12 contains no land cells
	Found 6708 nodata values
	
GWStore
	nodata_description: HUC12 contain no WRF-Hydro flowlines.
	Found 1746660 nodata values
	
Precip
	nodata_description: HUC12 contains no land cells
	Found 6708 nodata values
	
PrecipLand
	nodata_description: HUC12 contains no land cells
	Found 6708 nodata values
	
Recharge
	nodata_description: HUC12 contains

In [16]:
# View the variable encodings
for var in out_ds.data_vars:
    print(var)
    for key,item in out_ds[var].encoding.items():
        print(f'    {key}: {item}')

yrmo
    dtype: <U7
CatchmentArea
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    contiguous: True
    chunksizes: None
    source: /glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_2.nc
    original_shape: (86617,)
    _FillValue: -9999.0
LandFraction
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    contiguous: True
    chunksizes: None
    source: /glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_2.nc
    original_shape: (86617,)
    _FillValue: -9999.0
total_gridded_area
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    conti

### Write to output netCDF file

In [17]:
%%time

print('  Writing output to {0}'.format(out_file))
out_ds.compute()
out_ds.to_netcdf(out_file, encoding=output_encoding)
out_ds.close()

  Writing output to /glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_nan2.nc
CPU times: user 889 ms, sys: 2.02 s, total: 2.91 s
Wall time: 12 s


### THE END

In [18]:
# View the variable encodings
for var in out_ds.data_vars:
    print(var)
    for key,item in out_ds[var].encoding.items():
        print(f'    {key}: {item}')

yrmo
    dtype: <U7
CatchmentArea
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    contiguous: True
    chunksizes: None
    source: /glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_2.nc
    original_shape: (86617,)
    _FillValue: -9999.0
LandFraction
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    contiguous: True
    chunksizes: None
    source: /glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/Water_Budget/huc12_monthly_wb_iwaa_wrfhydro_WY1980_2022_2.nc
    original_shape: (86617,)
    _FillValue: -9999.0
total_gridded_area
    dtype: float32
    zlib: False
    szip: False
    zstd: False
    bzip2: False
    blosc: False
    shuffle: False
    complevel: 0
    fletcher32: False
    conti

In [ ]:
out_ds.isel({'huc_id':1500})['Precip'].plot(figsize=(12,5))

In [ ]:
from ipywidgets import interactive
%matplotlib inline

zone_name = 'huc_id' 

def plot_var(Variable):
    print(Variable)
    # Subset to just one HUC
    plot_ts = out_ds.where(out_ds[zone_name]==HUC, drop=True)[Variable]
        
    # Plot the time-series
    #plot_ts.attrs['long_name'] = varLabels[Variable]
    plot_ts.plot(aspect=2, figsize=(18, 6))
    
    # Plot the annual cycle
    plot_gpy = plot_ts.groupby('time.month').mean("time")
    #plot_gpy.attrs['long_name'] = varLabels[Variable]
    plot_gpy.plot(aspect=2, figsize=(18, 6))

# Variable and HUC options
plot_vars = ['avgSOILM_wltadj_depthmean',
             'avgSOILSAT_wltadj_top1',
             'Baseflow',
             'ET',
             'GWStore',
             'Precip',
             'PrecipLand',
             'Recharge',
             'Snowfall',
             'SoilSat',
             'SoilWater',
             'Surfaceflow',
             'SWE']

HUC = out_ds[zone_name].data[10000]
#HUC = 31002070303

# Plotting
interactive_plot = interactive(plot_var, Variable=plot_vars)
interactive_plot

## Exploration of the dataset to answer other questions that came up

## Find out what we did with the duplicate geometries

In [ ]:
# Create a dataframe from the input HUC12 mapping file
Mapping_File = r'/glade/derecho/scratch/ksampson/USGS/CONUS_Water_Budget/HUCs/Final_HUC12IDs.csv'
df = pd.read_csv(Mapping_File, index_col=[0])
df

### Find all non-unique HUC12 values

In [ ]:
#gdf[gdf.duplicated(['HUC12'], keep=False)]
non_unique = gdf[gdf.duplicated(['HUC12'], keep=False)]['HUC12'].unique()
non_unique

In [ ]:
# Make sure they are in the IWAAs Water Budget file
ds.where(ds['huc_id'].isin(non_unique.astype(ds['huc_id'].dtype)), drop=True)